# Package installation

In [1]:
# !pip install -r requirements.txt

# Imports

In [1]:
import pandas as pd
import langdetect
import datetime
import time

import webscrap
import cleanup 
import statdes

# Webscrap.py

In [5]:
url_ecb_download = "https://www.ecb.europa.eu/press/key/html/downloads.en.html"
url = "https://www.ecb.europa.eu" + webscrap.get_dataset_url(url_ecb_download)

df = pd.read_csv(url, sep = "|", encoding = 'utf-8')

KeyboardInterrupt: 

# Cleanup.py

In [ ]:
df = cleanup.clean_missing_data(df)
df = cleanup.keep_english(df)
df = cleanup.add_date(df)

In [ ]:
# cleanup.todisk(df)
df = cleanup.fromdisk()

# Statdes.py

In [ ]:
statdes.count_speeches(df)

In [ ]:
statdes.count_speeches(df, kind = 'bar')

In [ ]:
statdes.count_speeches(df, groupingby = 'Month', kind = 'bar')

In [ ]:
statdes.count_speeches(df, groupingby = 'Day', kind = 'bar')

In [ ]:
statdes.count_speeches(df, groupingby = 'DayofWeek', kind = 'bar')

In [ ]:
statdes.speech_months(df)

In [ ]:
statdes.distinct_dates(df)

In [ ]:
statdes.top_speakers(df)

# NLP

KeyError: 'plotly'